### Extract MARC Aerosol Data

Modified from `../aerosol_dists`, this will automate the extraction of the aerosol data for the greedy activation calculations: first, using NCO operators and the same pipeline as before, then using `xray` for post-processing down to the actual simulation parameter set.

In [1]:
## Copy the ncap2 scripts from aerosol_dists/
# YOU SHOULDN'T DO THIS ANYMORE - I FIXED A TYPO IN THE 
# COORD ORDER FOR 'P'
#!cp ../aerosol_dists/*nco .

---

This is the shell script for automating the extraction

**1) Configuration**

In [10]:
from subprocess import call
import glob, os, shutil

%run ../case_setup.py
pwd = os.getcwd()

#: Number of years to omit from beginning of model run
year_start = 5

#: Levels to extract
lev_bnds = [23, 26]

#: Latitude bounds
lat_bnds = [-60, 60]

#: Instant (h1) or averaged (h0)
time_mean = "instant"
time_str = "h1" if time_mean == "instant" else "h0"

#: Save_dir
save_dir = os.path.join(pwd, "data")

#: Final file name
fn_final_stem = "%s.new_aerosols.nc"

aer_mode_names = [
     "NUC", "AIT", "ACC", # sulfates
     "BC", "MBS",  # black carbon
     "OC", "MOS",  # organic carbon
]
number_vars =  ["n%s" % name for name in aer_mode_names]
mass_vars = ["m%s" % name for name in aer_mode_names]
size_vars = ["mu%s" % name for name in aer_mode_names]
act_vars = ["n%sACT" % name for name in aer_mode_names]
nc_vars = ["T", "Q","time_bnds","WSUB",
           "hyam", "hybm", "P0", "PS",]
ratios = ["BIM", "OIM"]
single_mode_vars = ["DST%02d" % i for i in xrange(1, 5)] + \
                   ["SSLT%02d" % i for i in xrange(1, 5)]

# List of all the vars necessary during processing
varnames = ",".join(nc_vars + number_vars + mass_vars + act_vars +
                    single_mode_vars + ["mOIM", "mBIM"])

# Final list of variable names
final_varnames = ",".join(nc_vars + 
                          ["P", "RHO", "kappaMOS", "nAERACT", ] + 
                          number_vars + size_vars + 
                          ["n%s" % s for s in single_mode_vars])

**2) Extraction**

In [11]:
%env OMP_NUM_THREADS = 30 
!echo using $OMP_NUM_THREADS threads

aer = "F2000" # only PD emissions

for act in CASES_ACT:
    print act, aer
    print "--"*20

    fn_final = fn_final_stem % act
    print "   Extracting aerosol-related variables -> %s" % fn_final

    extr_str = "-v %s " % varnames
    fn_final = fn_final_stem % act

    suffix = ""
    if lev_bnds: 
        lo, hi = lev_bnds
        suffix += "_lev-%d-%d" % (lo, hi)
        extr_str += " -d lev,%d,%d" % (lo, hi)
    if lat_bnds:
        lo, hi = lat_bnds
        suffix += "_lat-%d-%d" % (lo, hi)
        extr_str += " -d lat,%3.1f,%3.1f" % (lo, hi)
            
    fn_extr = "%s_%s_aerosol_monthly%s.nc" % (act, aer, suffix)
    # note we use the instantaneous fields
    !ncrcat -O {extr_str} {case_path(act,aer)}/{act}.cam2.{time_str}.00[0,1][0,{year_start}-9]-*.nc {save_dir}/{fn_extr} 

    print "   Computing pressure and density for unit conversions"
    fn_temp = "%s_%s_temp.nc" % (act, aer)
    !ncap2 -O -C -S {os.path.join(pwd, "extract_thermo.nco")} {save_dir}/{fn_extr} {save_dir}/{fn_temp} 
    !mv {save_dir}/{fn_temp} {save_dir}/{fn_extr}

    print "   Converting units on aerosol size dist parameters"
    !ncap2 -O -S {os.path.join(pwd, "aero_calcs.nco")} {save_dir}/{fn_extr} {save_dir}/{fn_temp} 
    !mv {save_dir}/{fn_temp} {save_dir}/{fn_extr}

    print "   Additional post-processing steps"
    print "      1) Total act"
    !ncap2 -O -s {"nAERACT="+"+".join(act_vars)} {save_dir}/{fn_extr} {save_dir}/{fn_temp} 
    !mv {save_dir}/{fn_temp} {save_dir}/{fn_extr}

    print "   Extracting final subset"
    !ncks -v {final_varnames} {save_dir}/{fn_extr} {save_dir}/{fn_temp}
    !mv {save_dir}/{fn_temp} {save_dir}/{fn_extr}
    
    print "   Renaming final file"
    !mv {save_dir}/{fn_extr} {save_dir}/{fn_final}

    print "   ... done."

    ## Some basic cleanup
    !rm -f {save_dir}/*.tmp

print "done with all cases\n"

env: OMP_NUM_THREADS=30
using 30 threads
pcm_min_smax F2000
----------------------------------------
   Extracting aerosol-related variables -> pcm_min_smax.aerosols.nc
   Computing pressure and density for unit conversions
   Converting units on aerosol size dist parameters
   Additional post-processing steps
      1) Total act
   Extracting final subset
   Renaming final file
   ... done.
pcm_comp F2000
----------------------------------------
   Extracting aerosol-related variables -> pcm_comp.aerosols.nc
   Computing pressure and density for unit conversions
   Converting units on aerosol size dist parameters
   Additional post-processing steps
      1) Total act
   Extracting final subset
   Renaming final file
   ... done.
arg_min_smax F2000
----------------------------------------
   Extracting aerosol-related variables -> arg_min_smax.aerosols.nc
   Computing pressure and density for unit conversions
   Converting units on aerosol size dist parameters
   Additional post-process

In [7]:
fn_final % 'test'

'test.aerosols.nc'